In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [ ]:
cx = sqlite3.connect("data/database.sqlite")

cs = cx.cursor()

countries = pd.read_sql_query("SELECT * FROM COUNTRY", con=cx)
matches = pd.read_sql_query("SELECT * FROM MATCH", con=cx)
players = pd.read_sql_query("SELECT * FROM PLAYER", con=cx)
teams = pd.read_sql_query("SELECT * FROM TEAM", con=cx)

In [ ]:
teams.sample(10)

In [ ]:
players.sample(10)

In [ ]:
list(matches.columns)


In [ ]:
matches.sample(10)


In [ ]:
matches["date_month"] = matches["date"].apply(
    lambda x: "-".join(x.split("-")[:2]))
n_matches_per_month = matches.groupby("date_month").agg({"id": "size"})

# plt.xticks()
# plt.xticks(np.arange(9), list(range(2008, 2017)))
fig, ax = plt.subplots()
ax.scatter(n_matches_per_month.index,
           n_matches_per_month["id"], marker='.', c='b')
for (i, l) in enumerate(ax.xaxis.get_ticklabels()):
    if i % 12 != 0:
        l.set_visible(False)
    else:
        l.set(rotation=45)

ax.set_title("Number of matches per month across years")
ax.set_ylabel("# matches")
plt.savefig("viz/n_matches_accross_years.png")



In [ ]:
players_columns = [f"home_player_{i}" for i in range(1, 12)] + \
    [f"away_player_{i}" for i in range(1, 12)] + \
    [f"home_player_X{i}" for i in range(1, 12)] + \
    [f"away_player_X{i}" for i in range(1, 12)] + \
    [f"home_player_Y{i}" for i in range(1, 12)] + \
    [f"away_player_Y{i}" for i in range(1, 12)]
matches["all_players"] = matches[players_columns].apply(
    lambda row: set([int(v) for v in row if not np.isnan(v) and not v < 12]), axis=1)

matches["all_players"].sample(10)

In [ ]:
import os
from pathlib import Path

file_to_save = Path("data/processed/number_of_matches_per_players.csv")
if os.path.exists(file_to_save):
    player_n_matches = pd.read_csv(file_to_save)
else:
    player_ids = []
    n_matches = []
    print("Counting number of matches per player([~2min])...")
    for i, p in tqdm(players.iterrows()):
        player_id = p["id"]
        m = matches["all_players"].apply(lambda se: player_id in se)
        player_ids.append(player_id)
        n_matches.append(matches[m].shape[0])
    print("OK")
    player_n_matches = pd.DataFrame({"id": player_ids, "n_matches": n_matches})
    player_n_matches = player_n_matches.join(players[["id", "player_name"]], rsuffix="_r") \
        .drop("id_r", axis=1) \
        .sort_values(by="n_matches", ascending=False)
    file_to_save.parent.mkdir(exist_ok=True, parents=True)
    player_n_matches.to_csv(file_to_save)
    print(f"Dataframe saved to {file_to_save} for further reuse.")


In [ ]:
N = 30
fig, ax = plt.subplots()
ax.bar(x=player_n_matches.head(N).player_name, height=player_n_matches.head(N).n_matches, width=0.5, color='b')
ax.set_title(f"Top {N} players by number of matches")
ax.set_ylabel("Total # matches")
for tick in ax.get_xticklabels():
    tick.set_rotation(90)
plt.tight_layout()
plt.savefig("viz/number_of_matches_per_player.png")
